In [4]:
import os
import cv2
import numpy as np

In [3]:
labels_Path = 'yolov3/coco.names'
labels = open(labels_Path).read().strip().split("\n")
labels[0:5]

['person', 'bicycle', 'car', 'motorbike', 'aeroplane']

In [4]:
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

In [8]:
configPath = 'yolov3/yolov3.cfg'
weightsPath = 'yolov3/yolov3.weights'

net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [14]:
cap = cv2.VideoCapture('video.mp4')
writer = None

while cap.isOpened():
    ret, image = cap.read()

    if not ret:
        break
    
    (H, W) = image.shape[:2]

    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    boxes = []
    confidences = []
    classIDs = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if confidence > 0.5:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            color = [int(c) for c in colors[classIDs[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(labels[classIDs[i]], confidences[i])
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, color, 2)
      
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"DIVX")
        writer = cv2.VideoWriter("OutputVideo.mp4", fourcc, 15,
        (W, H), True)
    
    cv2.imshow("Video", image)
    writer.write(image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break


writer.release()
cap.release()
cv2.destroyAllWindows()